<a href="https://colab.research.google.com/github/profteachkids/CHE2064_Spring2022/blob/main/AdiabaticFlash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/che.ipynb
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/FlattenWrap.ipynb
!pip install importnb

In [2]:
from importnb import Notebook
with Notebook(): 
    from FlattenWrap import DotDict, Range, RangeArray, Comp, CompArray, dtox
    from che import Props

import che
import numpy as np
from scipy.optimize import root

In [3]:
p = Props(['Ethanol','Isopropanol', 'Water'])

In [4]:
# Static parameters (Total feed, feed mole fractions, feed temperature and )
c=DotDict()
c.Ftot=10 # Total Feed moles
c.Fz = np.array([1/3, 1/3, 1/3]) # Equimolar feed composition
c.FT = 450 # Feed temperature
c.flashP= 101325 # Flash drum pressure

c.Vy = Comp(c.Fz) # Guess vapor/liquid composition equal to feed
c.Lx = Comp(c.Fz) # Comp - constrains mole fractions to behave like mole fractions!
c.flashT = Range(360, 273.15, c.FT)  # Guess and bounds for flash temperature
c.Vtot = Range(c.Ftot/2, 0., c.Ftot)  # Guess half of feed in vapor
c.Ltot = Range(c.Ftot/2, 0., c.Ftot)

In [5]:
wrap, x, d2, xtod, xtodunk = dtox(c)

In [6]:
d2

{'FT': 450,
 'Ftot': 10,
 'Fz': array([0.33333333, 0.33333333, 0.33333333]),
 'Ltot': array(5.),
 'Lx': array([0.33333333, 0.33333333, 0.33333333]),
 'Vtot': array(5.),
 'Vy': array([0.33333333, 0.33333333, 0.33333333]),
 'flashP': 101325,
 'flashT': array(360.)}

In [7]:
def eqs(d):
    r=DotDict()
    V = d.Vy * d.Vtot # Moles of each component = mole fractions * total moles
    L = d.Lx * d.Ltot
    F = d.Fz * d.Ftot
    r.mass_balance = F - V - L # Mass balance for each component (vectors!)

    # Hmix calculates the enthalpy given the temperature and moles of each
    # component in the vapor and liquid phases
    FH = p.Hl(nL=F, T=d.FT)
    VH = p.Hv(nV=V, T=d.flashT)
    LH = p.Hl(nL=L, T=d.flashT)
    r.energy_balance = (FH - VH - LH)

    # Raoults with NRTL activity coefficient correction.  One-liner!
    fugL = d.Lx  * p.NRTL_gamma(d.Lx,d.flashT)* p.Pvap(d.flashT)
    fugV = d.Vy * d.flashP
    r.VLE = fugL - fugV
    return np.concatenate([r.mass_balance, np.atleast_1d(r.energy_balance), r.VLE]), r 


In [8]:
eqs_wrapped = wrap(eqs)

In [9]:
sol=root(eqs_wrapped, x)
sol

    fjac: array([[-2.93233152e-05,  1.53805921e-05,  1.39427232e-05,
        -6.47949621e-01, -6.22484674e-01,  2.98555445e-01,
         3.21774400e-01],
       [-1.26269048e-05, -1.90384605e-05,  3.16653657e-05,
         4.33725798e-01, -2.83916113e-01, -4.40572575e-01,
         7.32918398e-01],
       [ 4.23241601e-05, -8.24812204e-06, -3.40760383e-05,
         6.25453258e-01, -4.73855106e-01,  5.86320175e-01,
        -2.01241670e-01],
       [ 6.88571594e-04,  2.57851494e-04, -9.46423092e-04,
        -2.91794613e-02, -5.54405422e-01, -6.10727945e-01,
        -5.64617683e-01],
       [-5.81789805e-01, -2.05229823e-01,  7.87019632e-01,
        -3.42308349e-06, -6.62205417e-04, -6.94203219e-04,
        -7.21154959e-04],
       [-6.06648583e-01,  7.54029712e-01, -2.51826703e-01,
         3.81491403e-05,  9.23353249e-06,  8.66737357e-06,
         6.22653908e-06],
       [-5.41754227e-01, -6.23955003e-01, -5.63189588e-01,
        -2.30465122e-06, -5.57812640e-07, -5.23610140e-07,
        

In [10]:
xtodunk(sol.x)

{'Ltot': array(6.02836122),
 'Lx': array([0.32122647, 0.32919072, 0.34958281]),
 'Vtot': array(3.97163878),
 'Vy': array([0.35170977, 0.33962121, 0.30866903]),
 'flashT': array(352.85497499)}

In [11]:
xtod(sol.x,d2)

{'FT': 450,
 'Ftot': 10,
 'Fz': array([0.33333333, 0.33333333, 0.33333333]),
 'Ltot': array(6.02836122),
 'Lx': array([0.32122647, 0.32919072, 0.34958281]),
 'Vtot': array(3.97163878),
 'Vy': array([0.35170977, 0.33962121, 0.30866903]),
 'flashP': 101325,
 'flashT': array(352.85497499)}